In [ ]:
%load_ext autoreload
%autoreload 2

from waffles.plotting.plot import plot_ChannelWsGrid
from waffles.utils.fit_peaks.fit_peaks import fit_peaks_of_ChannelWsGrid
from waffles.utils.integral.WindowIntegrator import WindowIntegrator
from waffles.utils.integral.integral_utils import get_pulse_window_limits
from waffles.np04_analysis.led_calibration.utils import compute_average_baseline_std
from waffles.np04_utils.utils import get_average_baseline_std_from_file
from waffles.data_classes.StoreWfAna import StoreWfAna
from waffles.utils.filtering_utils import coarse_selection_for_led_calibration
from waffles.utils.baseline.baseline_utils import subtract_baseline
from waffles.data_classes.IPDict import IPDict
from waffles.utils.baseline.WindowBaseliner import WindowBaseliner
from waffles.data_classes.Waveform import Waveform
from waffles.data_classes.WaveformSet import WaveformSet
from waffles.data_classes.UniqueChannel import UniqueChannel
from waffles.data_classes.ChannelWsGrid import ChannelWsGrid
from waffles.coldboxVD.utils.spybuffer_reader import create_waveform_set_from_spybuffer
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
membrane = 'M3'
channel = 29
bias = '31V47' #4.5 OV
vgain = 1700
coldbox_folder = "/eos/experiment/neutplatform/protodune/experiments/ColdBoxVD/November2025run/spy_buffer/VGAIN_SCAN"
input_file = f"{coldbox_folder}/{membrane}/vgain_scan_{membrane}_DVbias_{bias}/vgain_{vgain}/channel_{channel}.dat"

In [ ]:
wfset = create_waveform_set_from_spybuffer(filename=input_file, WFs=40000, length=1024, config_channel=channel)


In [ ]:
wf_index = 1
plt.figure()
plt.plot(np.array(range(0,1024))*1.6, wfset.waveforms[wf_index].adcs, label="Waveform 0")
plt.xlabel("Time (ns)")
plt.ylabel("Adcs")
plt.title(f"Waveform index: {wf_index}")
plt.show()

In [ ]:
baseliner_input_parameters = IPDict({
            'baseline_limits': (0,350),
            'std_cut': 3.,
            'type': 'mean'
        })

checks_kwargs = IPDict({
    'points_no': wfset.points_per_wf
})

baseline_analysis_label = 'baseline'

_ = wfset.analyse(
    baseline_analysis_label,
    WindowBaseliner,
    baseliner_input_parameters,
    checks_kwargs=checks_kwargs,
    overwrite=True
)

In [ ]:
wfset.apply(
        subtract_baseline,
        baseline_analysis_label,
        show_progress=False
    )

In [ ]:
wf_index = 19998
plt.figure()
plt.plot(np.array(range(0,1024))*1.6, wfset.waveforms[wf_index].adcs, label="Waveform 0")
plt.xlabel("Time (ns)")
plt.ylabel("Adcs")
plt.title(f"Waveform index: {wf_index}")
plt.show()

In [ ]:
null_baseline_analysis_label = 'null_baseliner'
_ = wfset.analyse(
            null_baseline_analysis_label,
            StoreWfAna,
            {'baseline': 0.},
            overwrite=True
        )

In [ ]:
print(len(wfset.waveforms))

In [ ]:
wfset_new = WaveformSet.from_filtered_WaveformSet(
            wfset,
            coarse_selection_for_led_calibration,
            null_baseline_analysis_label,
            30,
            30
        )

print(len(wfset_new.waveforms))

In [ ]:
average_baseline_std = compute_average_baseline_std(
                        wfset_new,
                        # What is taken from the baseline analysis here is
                        # the baseline STD (which is the same after baseline
                        # subtraction), not the baseline value itself
                        baseline_analysis_label
                    )

In [ ]:
def baseline_std_selection(waveform: Waveform,
    baseline_analysis_label: str,
    mean_std: float,
    n_std: float = 3.0
    ) -> bool:
    if waveform.analyses[baseline_analysis_label].result['baseline_std'] < mean_std*n_std:
        return True
    else:
        return False

In [ ]:
wfset_cut = WaveformSet.from_filtered_WaveformSet(
            wfset_new,
            baseline_std_selection,
            baseline_analysis_label,
            average_baseline_std,
            1
        )
print(len(wfset.waveforms))
print(len(wfset_new.waveforms))
print(len(wfset_cut.waveforms))

In [ ]:
mean_wf = wfset_cut.compute_mean_waveform()

plt.figure()
plt.plot(np.array(range(0,1024))*1.6, mean_wf.adcs, label="Mean wf")
plt.xlabel("Time (ns)")
plt.ylabel("Adcs")
plt.title(f"Mean waveform")
plt.show()

In [ ]:
deviation_from_baseline = 0.3

aux_limits = get_pulse_window_limits(
                    adcs_array = -mean_wf.adcs,
                    baseline = 0,
                    deviation_from_baseline = deviation_from_baseline,
                    get_zero_crossing_upper_limit = False
                )
print(aux_limits)

In [ ]:
federico_limits = (382, 406)

plt.figure()
plt.plot(np.array(range(0,1024)), mean_wf.adcs, label="Mean wf")
plt.axvline(
    aux_limits[0],
    linestyle="--",
    color="red",
    linewidth=1,
    label=f"My LL = {aux_limits[0]} ({deviation_from_baseline:.1f} σ)"
)

plt.axvline(
    aux_limits[1],
    linestyle="--",
    color="blue",
    linewidth=1,
    label=f"My UL = {aux_limits[1]} ({deviation_from_baseline:.1f} σ)"
)

plt.axvline(
    federico_limits[0],
    linestyle="-",
    color="red",
    linewidth=1,
    label=f"Federico LL = {federico_limits[0]}"
)

plt.axvline(
    federico_limits[1],
    linestyle="-",
    color="blue",
    linewidth=1,
    label=f"Federico UL = {federico_limits[1]}"
)

plt.legend()
plt.xlabel("Time ticks (AU)")
plt.ylabel("Adcs")
plt.title(f"Mean waveform")
plt.xlim(350,500)
plt.show()

In [ ]:
aux_limits = federico_limits

integration_analysis_label = 'integration_analysis'

integrator_input_parameters = IPDict({
        'baseline_analysis': null_baseline_analysis_label,
        'inversion': False,
        'int_ll': aux_limits[0],
        'int_ul': aux_limits[1],
        'amp_ll': aux_limits[0],
        'amp_ul': aux_limits[1]
    })

checks_kwargs = IPDict({
                    'points_no': wfset_cut.points_per_wf
                })

_ = wfset_cut.analyse(
    integration_analysis_label,
    WindowIntegrator,
    integrator_input_parameters,
    checks_kwargs=checks_kwargs,
    overwrite=True
)

In [ ]:
from waffles.np02_utils.PlotUtils import np02_gen_grids
from waffles.data_classes.UniqueChannel import UniqueChannel

kwargs={}

detector=[ UniqueChannel(10, x) for x in [29] ]
grid= np02_gen_grids(wfset_cut, detector, rows=kwargs.pop("rows", 0), cols=kwargs.pop("cols", 0))


In [ ]:
grid['Custom'].compute_calib_histos(
            bins_number=100,
            domain=np.array([-1000,10000]),
            variable='integral',
            analysis_label=integration_analysis_label
        )

In [ ]:
fit_peaks_of_ChannelWsGrid( 
            grid['Custom'],
            max_peaks = 3,
            prominence = 0.15,
            initial_percentage = 0.10,
            percentage_step = 0.05,
            fit_type = 'correlated_gaussians'

        )

In [ ]:
figure = plot_ChannelWsGrid(
            grid['Custom'],
            figure=None,
            share_x_scale=False,
            share_y_scale=False,
            mode="calibration",
            wfs_per_axes=None,
            plot_peaks_fits=True
        )
figure.show()